In [ ]:
import numpy as np
import pandas as pd

class InverseDifferencing:
    """
    بازگردانی تفاضل‌گیری در سری زمانی.

    Parameters
    ----------
    method : str, default='cumsum'
        روش بازگردانی (فعلاً فقط 'cumsum' پشتیبانی می‌شود).
    config : dict, optional
        تنظیمات:
            - order : int, default=1
                مرتبه تفاضل‌گیری.
            - initial_values : list or dict, optional
                مقادیر اولیه برای هر ستون. اگر دیتافریم است، باید به تعداد ستون‌ها یا به صورت دیکشنری {ستون: مقدار} باشد.
            - columns : list, optional
                نام ستون‌هایی که باید بازگردانی شوند. اگر None، همه ستون‌های عددی.
    """
    def __init__(self, method='cumsum', config=None):
        self.method = method
        self.config = config or {}
        self.columns = self.config.get('columns', None)
        self.order = self.config.get('order', 1)
        self.initial_values = self.config.get('initial_values', None)
        self.fitted_columns_ = None

    def fit(self, X, y=None):
        """بررسی وجود ستون‌ها و تطابق initial_values."""
        if isinstance(X, pd.DataFrame):
            if self.columns is None:
                self.fitted_columns_ = X.select_dtypes(include=[np.number]).columns.tolist()
            else:
                self.fitted_columns_ = [col for col in self.columns if col in X.columns]
                missing = set(self.columns) - set(self.fitted_columns_)
                if missing:
                    raise ValueError(f"Columns not found: {missing}")
        else:
            # اگر X آرایه است، ستون‌ها را با اندیس فرض می‌کنیم
            n_features = X.shape[1] if X.ndim > 1 else 1
            if self.columns is None:
                self.fitted_columns_ = list(range(n_features))
            else:
                self.fitted_columns_ = self.columns

        # بررسی initial_values
        if self.initial_values is not None:
            if isinstance(self.initial_values, (int, float)):
                # یک مقدار برای همه ستون‌ها
                self.initial_values = [self.initial_values] * len(self.fitted_columns_)
            elif isinstance(self.initial_values, dict):
                # تبدیل به لیست به ترتیب ستون‌ها
                self.initial_values = [self.initial_values.get(col, None) for col in self.fitted_columns_]
            elif isinstance(self.initial_values, (list, np.ndarray)):
                if len(self.initial_values) != len(self.fitted_columns_):
                    raise ValueError("Length of initial_values must match number of columns.")
            else:
                raise TypeError("initial_values must be a number, dict, or list.")
        return self

    def transform(self, X):
        """اعمال بازگردانی تفاضل."""
        if not hasattr(self, 'fitted_columns_'):
            raise RuntimeError("Fit must be called before transform.")

        X_out = X.copy() if isinstance(X, pd.DataFrame) else np.copy(X)
        for i, col in enumerate(self.fitted_columns_):
            # استخراج سری
            if isinstance(X_out, pd.DataFrame):
                series = X_out[col].values
            else:
                series = X_out[:, col] if X_out.ndim > 1 else X_out[:]

            # بازگردانی با تجمعی
            if self.method == 'cumsum':
                # نیاز به مقادیر اولیه
                if self.initial_values is not None:
                    init = self.initial_values[i]
                    # برای تفاضل مرتبه d، باید d مقدار اولیه داشته باشیم؟
                    # فعلاً فرض می‌کنیم initial_values شامل مقدار قبل از اولین تفاضل است
                    # برای مرتبه 1: restored[0] = init + series[0]
                    restored = np.zeros_like(series)
                    restored[0] = init + series[0] if init is not None else series[0]
                    for t in range(1, len(series)):
                        restored[t] = restored[t-1] + series[t]
                else:
                    # اگر initial_values داده نشده، از صفر شروع می‌کنیم
                    restored = np.cumsum(series)
            else:
                raise ValueError(f"Method '{self.method}' not supported.")

            # قرار دادن در خروجی
            if isinstance(X_out, pd.DataFrame):
                X_out[col] = restored
            else:
                if X_out.ndim > 1:
                    X_out[:, col] = restored
                else:
                    X_out[:] = restored

        return X_out

In [ ]:
class Smoothing:
    """
    هموارسازی سری زمانی.

    Parameters
    ----------
    method : str, default='moving_average'
        'moving_average' یا 'exponential'.
    config : dict, optional
        تنظیمات:
            - window : int, default=3
                اندازه پنجره برای میانگین متحرک.
            - alpha : float, default=0.3
                ضریب هموارسازی نمایی (بین ۰ و ۱).
            - min_periods : int, optional
                حداقل تعداد داده برای محاسبه (برای ابتدای سری).
            - center : bool, default=False
                آیا پنجره مرکزی باشد یا به سمت چپ.
            - columns : list, optional
                ستون‌های هدف.
            - suffix : str, default='_smoothed'
                پسوند برای ستون‌های جدید.
    """
    def __init__(self, method='moving_average', config=None):
        self.method = method
        self.config = config or {}
        self.columns = self.config.get('columns', None)
        self.window = self.config.get('window', 3)
        self.alpha = self.config.get('alpha', 0.3)
        self.min_periods = self.config.get('min_periods', 1)
        self.center = self.config.get('center', False)
        self.suffix = self.config.get('suffix', '_smoothed')
        self.fitted_columns_ = None

    def fit(self, X, y=None):
        """تعیین ستون‌های هدف."""
        if isinstance(X, pd.DataFrame):
            if self.columns is None:
                self.fitted_columns_ = X.select_dtypes(include=[np.number]).columns.tolist()
            else:
                self.fitted_columns_ = [col for col in self.columns if col in X.columns]
                missing = set(self.columns) - set(self.fitted_columns_)
                if missing:
                    raise ValueError(f"Columns not found: {missing}")
        else:
            n_features = X.shape[1] if X.ndim > 1 else 1
            if self.columns is None:
                self.fitted_columns_ = list(range(n_features))
            else:
                self.fitted_columns_ = self.columns
        return self

    def transform(self, X):
        """اعمال هموارسازی و افزودن ستون‌های جدید."""
        X_out = X.copy() if isinstance(X, pd.DataFrame) else np.copy(X)
        for col in self.fitted_columns_:
            if isinstance(X_out, pd.DataFrame):
                series = X_out[col]
            else:
                series = X_out[:, col] if X_out.ndim > 1 else X_out[:]

            if self.method == 'moving_average':
                smoothed = series.rolling(window=self.window, min_periods=self.min_periods, center=self.center).mean()
            elif self.method == 'exponential':
                # استفاده از pandas ewm
                smoothed = series.ewm(alpha=self.alpha, adjust=False).mean()
            else:
                raise ValueError(f"Method '{self.method}' not supported.")

            # اضافه کردن به عنوان ستون جدید
            if isinstance(X_out, pd.DataFrame):
                new_col = f"{col}{self.suffix}"
                X_out[new_col] = smoothed
            else:
                # برای آرایه، ستون جدید اضافه می‌کنیم (باعث تغییر شکل می‌شود)
                # بهتر است آرایه را به DataFrame تبدیل کنیم یا با احتیاط کار کنیم.
                # در اینجا فرض می‌کنیم X DataFrame است.
                raise NotImplementedError("Smoothing on numpy arrays not fully implemented; please use DataFrame.")
        return X_out

In [ ]:
from scipy import stats
from statsmodels.tsa.stattools import acf, pacf
import warnings

class ResidualDiagnostics:
    """
    تشخیص باقیمانده‌ها و محاسبه آماره‌ها.

    Parameters
    ----------
    method : str, default='residuals'
        فعلاً فقط 'residuals' پشتیبانی می‌شود.
    config : dict, optional
        تنظیمات:
            - prediction_column : str
                نام ستون حاوی پیش‌بینی.
            - true_column : str
                نام ستون حاوی مقادیر واقعی.
            - residual_column : str, default='residual'
                نام ستون خروجی برای باقیمانده‌ها.
            - tests : list, default=['normality', 'acf']
                لیست تست‌ها: 'normality', 'acf', 'pacf', 'box_pierce', 'ljung_box'
            - lags : int, optional
                تعداد لگ‌ها برای تست‌های خودهمبستگی.
    """
    def __init__(self, method='residuals', config=None):
        self.method = method
        self.config = config or {}
        self.pred_col = self.config.get('prediction_column')
        self.true_col = self.config.get('true_column')
        self.residual_col = self.config.get('residual_column', 'residual')
        self.tests = self.config.get('tests', ['normality', 'acf'])
        self.lags = self.config.get('lags', 10)
        self.results_ = {}  # برای ذخیره نتایج تست‌ها

    def fit(self, X, y=None):
        """بررسی وجود ستون‌های مورد نیاز."""
        if self.pred_col is None or self.true_col is None:
            raise ValueError("prediction_column and true_column must be specified in config.")
        if not isinstance(X, pd.DataFrame):
            raise TypeError("X must be a pandas DataFrame.")
        if self.pred_col not in X.columns:
            raise ValueError(f"Prediction column '{self.pred_col}' not found.")
        if self.true_col not in X.columns:
            raise ValueError(f"True column '{self.true_col}' not found.")
        return self

    def transform(self, X):
        """محاسبه باقیمانده‌ها و انجام تست‌ها."""
        X_out = X.copy()
        residuals = X_out[self.true_col] - X_out[self.pred_col]
        X_out[self.residual_col] = residuals

        # انجام تست‌های تشخیصی
        self.results_ = {}
        res_vals = residuals.dropna().values

        if 'normality' in self.tests and len(res_vals) > 3:
            # تست شاپیرو-ویلک
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                shapiro_stat, shapiro_p = stats.shapiro(res_vals)
            self.results_['shapiro'] = {'statistic': shapiro_stat, 'p_value': shapiro_p}

        if 'acf' in self.tests and len(res_vals) > self.lags:
            acf_vals = acf(res_vals, nlags=self.lags, fft=False)
            self.results_['acf'] = acf_vals.tolist()

        if 'pacf' in self.tests and len(res_vals) > self.lags:
            pacf_vals = pacf(res_vals, nlags=self.lags)
            self.results_['pacf'] = pacf_vals.tolist()

        if 'box_pierce' in self.tests and len(res_vals) > self.lags:
            from statsmodels.stats.diagnostic import acorr_ljungbox
            result = acorr_ljungbox(res_vals, lags=[self.lags], boxpierce=True, return_df=False)
            self.results_['box_pierce'] = {
                'statistic': result[0]['bp_stat'].values[0] if hasattr(result[0], 'bp_stat') else None,
                'p_value': result[0]['bp_pvalue'].values[0] if hasattr(result[0], 'bp_pvalue') else None
            }

        if 'ljung_box' in self.tests and len(res_vals) > self.lags:
            from statsmodels.stats.diagnostic import acorr_ljungbox
            result = acorr_ljungbox(res_vals, lags=[self.lags], boxpierce=False, return_df=False)
            self.results_['ljung_box'] = {
                'statistic': result[0]['lb_stat'].values[0] if hasattr(result[0], 'lb_stat') else None,
                'p_value': result[0]['lb_pvalue'].values[0] if hasattr(result[0], 'lb_pvalue') else None
            }

        return X_out

    def get_results(self):
        """بازگرداندن نتایج تست‌ها."""
        return self.results_

In [ ]:
class ConfidenceInterval:
    """
    محاسبه بازه اطمینان برای پیش‌بینی‌ها.

    Parameters
    ----------
    method : str, default='normal'
        'normal', 'bootstrap', 'quantile'.
    config : dict, optional
        تنظیمات:
            - prediction_column : str
                نام ستون حاوی پیش‌بینی.
            - residual_column : str, optional
                نام ستون حاوی باقیمانده‌ها (برای روش نرمال و بوت‌استرپ). اگر نباشد، از انحراف معیار پیش‌بینی‌ها استفاده می‌شود.
            - confidence_level : float, default=0.95
                سطح اطمینان.
            - lower_column : str, default='lower_bound'
                نام ستون خروجی برای کران پایین.
            - upper_column : str, default='upper_bound'
                نام ستون خروجی برای کران بالا.
            - n_bootstrap : int, default=1000
                تعداد نمونه‌های بوت‌استرپ (برای روش bootstrap).
            - quantiles : list, optional
                چندک‌های دلخواه (برای روش quantile). اگر None، از confidence_level استفاده می‌شود.
    """
    def __init__(self, method='normal', config=None):
        self.method = method
        self.config = config or {}
        self.pred_col = self.config.get('prediction_column')
        self.residual_col = self.config.get('residual_column', None)
        self.confidence_level = self.config.get('confidence_level', 0.95)
        self.lower_col = self.config.get('lower_column', 'lower_bound')
        self.upper_col = self.config.get('upper_column', 'upper_bound')
        self.n_bootstrap = self.config.get('n_bootstrap', 1000)
        self.quantiles = self.config.get('quantiles', None)
        self.fitted_ = False

    def fit(self, X, y=None):
        """بررسی ستون‌ها و محاسبه آماره‌های مورد نیاز."""
        if self.pred_col is None:
            raise ValueError("prediction_column must be specified in config.")
        if not isinstance(X, pd.DataFrame):
            raise TypeError("X must be a pandas DataFrame.")
        if self.pred_col not in X.columns:
            raise ValueError(f"Prediction column '{self.pred_col}' not found.")

        # اگر residual_column داده شده، آن را بررسی می‌کنیم
        if self.residual_col is not None and self.residual_col not in X.columns:
            raise ValueError(f"Residual column '{self.residual_col}' not found.")

        # محاسبه آماره‌ها بر اساس روش
        if self.method == 'normal':
            # محاسبه میانگین و انحراف معیار باقیمانده‌ها (یا پیش‌بینی‌ها)
            if self.residual_col:
                residuals = X[self.residual_col].dropna()
                self.residual_std_ = residuals.std()
                self.residual_mean_ = residuals.mean()
            else:
                # اگر باقیمانده نداریم، از خود پیش‌بینی‌ها؟ معمولاً نمی‌شود.
                # در عوض می‌توان از خطای استاندارد پیش‌بینی استفاده کرد.
                # در اینجا ساده‌سازی: انحراف معیار پیش‌بینی‌ها
                preds = X[self.pred_col].dropna()
                self.residual_std_ = preds.std()
                self.residual_mean_ = 0
        elif self.method == 'bootstrap':
            # نیاز به داده برای نمونه‌گیری مجدد
            if self.residual_col is None:
                raise ValueError("bootstrap method requires residual_column to resample.")
            self.residuals_ = X[self.residual_col].dropna().values
        elif self.method == 'quantile':
            if self.quantiles is None:
                alpha = 1 - self.confidence_level
                self.quantiles = [alpha/2, 1 - alpha/2]
        else:
            raise ValueError(f"Method '{self.method}' not supported.")

        self.fitted_ = True
        return self

    def transform(self, X):
        """محاسبه کران‌های بازه اطمینان برای هر نمونه."""
        if not self.fitted_:
            raise RuntimeError("Fit must be called before transform.")
        if self.pred_col not in X.columns:
            raise ValueError(f"Prediction column '{self.pred_col}' not found in transform data.")

        X_out = X.copy()
        preds = X_out[self.pred_col].values

        if self.method == 'normal':
            # فاصله بر اساس توزیع نرمال
            z = stats.norm.ppf(1 - (1 - self.confidence_level) / 2)
            margin = z * self.residual_std_
            lower = preds - margin
            upper = preds + margin
        elif self.method == 'bootstrap':
            # برای هر نمونه، با جایگذاری مجدد از باقیمانده‌ها نمونه می‌گیریم و چندک محاسبه می‌کنیم
            # این روش می‌تواند سنگین باشد. راه ساده‌تر: ساختن توزیع bootstrap برای هر نمونه؟
            # در عمل، معمولاً بازه‌های bootstrap را با نمونه‌گیری از باقیمانده‌ها و ساختن پیش‌بینی‌های جدید محاسبه می‌کنند.
            # در اینجا یک پیاده‌سازی ساده: برای هر نمونه، n_bootstrap بار یک باقیمانده تصادفی را به پیش‌بینی اضافه می‌کنیم.
            n = len(preds)
            lower = np.zeros(n)
            upper = np.zeros(n)
            for i in range(n):
                boot_samples = preds[i] + np.random.choice(self.residuals_, size=self.n_bootstrap, replace=True)
                lower[i] = np.percentile(boot_samples, (1 - self.confidence_level) / 2 * 100)
                upper[i] = np.percentile(boot_samples, (1 + self.confidence_level) / 2 * 100)
        elif self.method == 'quantile':
            # استفاده از چندک‌های مشخص
            # این روش نیاز به محاسبه چندک‌ها از توزیع پیش‌بینی‌ها دارد، اما اینجا فقط بر اساس داده موجود است.
            # اگر چندک‌ها ثابت باشند، برای همه نمونه‌ها یکسان خواهند بود.
            # اما کاربر می‌تواند چندک‌هایی مانند [0.025, 0.975] بدهد.
            q_low, q_high = self.quantiles[0], self.quantiles[-1]
            # از توزیع پیش‌بینی‌ها؟ یا از باقیمانده‌ها؟
            # ساده: فرض می‌کنیم پیش‌بینی‌ها نرمال هستند و از چندک‌های نرمال استفاده می‌کنیم.
            # اما این در واقع همان روش normal است.
            # برای تنوع، می‌توان از چندک‌های تجربی باقیمانده‌ها استفاده کرد.
            if self.residual_col is not None and self.residual_col in X.columns:
                residuals = X[self.residual_col].values
                q_low_val = np.percentile(residuals, q_low * 100)
                q_high_val = np.percentile(residuals, q_high * 100)
                lower = preds + q_low_val
                upper = preds + q_high_val
            else:
                # از توزیع نرمال با انحراف معیار پیش‌بینی‌ها
                std = np.std(preds)
                z_low = stats.norm.ppf(q_low)
                z_high = stats.norm.ppf(q_high)
                lower = preds + z_low * std
                upper = preds + z_high * std
        else:
            raise ValueError(f"Method '{self.method}' not supported.")

        X_out[self.lower_col] = lower
        X_out[self.upper_col] = upper
        return X_out

In [ ]:
class OutlierDetectionOnPredictions:
    """
    تشخیص نقاط پرت در ستون پیش‌بینی (یا هر ستون عددی).

    Parameters
    ----------
    method : str, default='zscore'
        روش تشخیص: 'zscore', 'iqr', 'isolation_forest', 'dbscan' و ...
    config : dict, optional
        تنظیمات:
            - column : str
                نام ستون هدف.
            - threshold : float, default=3.0
                آستانه برای zscore.
            - iqr_multiplier : float, default=1.5
                ضریب IQR.
            - contamination : float, default=0.1
                نسبت outlier برای isolation forest.
            - random_state : int, default=42
                seed.
            - flag_column : str, default='is_outlier'
                نام ستون خروجی بولین.
            - action : str, optional
                'flag', 'remove', 'replace' (فعلاً فقط 'flag' پشتیبانی می‌شود).
    """
    def __init__(self, method='zscore', config=None):
        self.method = method
        self.config = config or {}
        self.column = self.config.get('column')
        self.threshold = self.config.get('threshold', 3.0)
        self.iqr_multiplier = self.config.get('iqr_multiplier', 1.5)
        self.contamination = self.config.get('contamination', 0.1)
        self.random_state = self.config.get('random_state', 42)
        self.flag_column = self.config.get('flag_column', 'is_outlier')
        self.action = self.config.get('action', 'flag')
        self.model_ = None
        self.fitted_ = False

    def fit(self, X, y=None):
        """یادگیری پارامترهای تشخیص (برای روش‌های مبتنی بر مدل)."""
        if self.column is None:
            raise ValueError("column must be specified in config.")
        if not isinstance(X, pd.DataFrame):
            raise TypeError("X must be a pandas DataFrame.")
        if self.column not in X.columns:
            raise ValueError(f"Column '{self.column}' not found.")

        data = X[self.column].dropna().values.reshape(-1, 1)

        if self.method == 'zscore':
            self.mean_ = np.mean(data)
            self.std_ = np.std(data)
        elif self.method == 'iqr':
            q1 = np.percentile(data, 25)
            q3 = np.percentile(data, 75)
            self.q1_ = q1
            self.q3_ = q3
            self.iqr_ = q3 - q1
        elif self.method == 'isolation_forest':
            from sklearn.ensemble import IsolationForest
            self.model_ = IsolationForest(contamination=self.contamination, random_state=self.random_state)
            self.model_.fit(data)
        elif self.method == 'dbscan':
            from sklearn.cluster import DBSCAN
            self.model_ = DBSCAN(eps=self.config.get('eps', 0.5), min_samples=self.config.get('min_samples', 5))
            self.model_.fit(data)
            # DBSCAN برچسب -1 را outlier می‌دهد
        else:
            raise ValueError(f"Method '{self.method}' not supported.")

        self.fitted_ = True
        return self

    def transform(self, X):
        """تشخیص outlier و افزودن پرچم."""
        if not self.fitted_:
            raise RuntimeError("Fit must be called before transform.")
        if self.column not in X.columns:
            raise ValueError(f"Column '{self.column}' not found in transform data.")

        X_out = X.copy()
        data = X_out[self.column].values.reshape(-1, 1)

        if self.method == 'zscore':
            z_scores = np.abs((data - self.mean_) / self.std_)
            outliers = z_scores.flatten() > self.threshold
        elif self.method == 'iqr':
            lower = self.q1_ - self.iqr_multiplier * self.iqr_
            upper = self.q3_ + self.iqr_multiplier * self.iqr_
            outliers = (data < lower) | (data > upper)
            outliers = outliers.flatten()
        elif self.method == 'isolation_forest':
            preds = self.model_.predict(data)
            outliers = preds == -1
        elif self.method == 'dbscan':
            preds = self.model_.fit_predict(data)  # باید دوباره fit کنیم؟ بهتر است در fit انجام شده باشد.
            outliers = preds == -1
        else:
            raise ValueError(f"Method '{self.method}' not supported.")

        X_out[self.flag_column] = outliers

        # اگر action 'replace' باشد، می‌توان outlierها را با میانگین جایگزین کرد
        if self.action == 'replace':
            replacement = np.mean(data[~outliers]) if np.any(~outliers) else 0
            X_out.loc[outliers, self.column] = replacement
        elif self.action == 'remove':
            X_out = X_out[~outliers].reset_index(drop=True)

        return X_out